In [ ]:
pip install keras-tuner

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.1/129.1 kB 3.3 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.optimizers import Adam
import datetime
from sklearn.preprocessing import MinMaxScaler

## Import data

In [ ]:
df = pd.read_csv('/content/2021-2022_bit.csv')

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 264959 entries, 0 to 264958
Data columns (total 7 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   Unnamed: 0  264959 non-null  int64  
 1   Date        264959 non-null  object 
 2   Open        264959 non-null  float64
 3   High        264959 non-null  float64
 4   Low         264959 non-null  float64
 5   Close       264959 non-null  float64
 6   Volume BTC  264959 non-null  float64
dtypes: float64(5), int64(1), object(1)
memory usage: 14.2+ MB


In [ ]:
df = df.iloc[:,1:].copy()

In [ ]:
# reverse the data
df = df[::-1]

In [ ]:
df.columns = ['date', 'open', 'high', 'low', 'close', 'vol']

In [ ]:
df['date'] = pd.to_datetime(df['date'])

In [ ]:
df = df.set_index('date')

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 264959 entries, 2021-08-01 00:01:00 to 2022-01-31 23:59:00
Data columns (total 5 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   open    264959 non-null  float64
 1   high    264959 non-null  float64
 2   low     264959 non-null  float64
 3   close   264959 non-null  float64
 4   vol     264959 non-null  float64
dtypes: float64(5)
memory usage: 12.1 MB


In [ ]:
df

,open,high,low,close,vol
date,,,,,
2021-08-01 00:01:00,41469.05,41469.05,41332.91,41348.38,3.513764
2021-08-01 00:02:00,41348.38,41348.38,41165.96,41196.18,8.040649
2021-08-01 00:03:00,41196.18,41339.38,41183.56,41327.59,3.081120
2021-08-01 00:04:00,41327.59,41387.71,41327.59,41379.84,0.044790
2021-08-01 00:05:00,41379.84,41550.50,41379.84,41550.50,0.196676
...,...,...,...,...,...
2022-01-31 23:55:00,38479.35,38510.17,38479.35,38510.17,0.008319
2022-01-31 23:56:00,38510.17,38561.36,38506.33,38519.38,4.325445
2022-01-31 23:57:00,38519.38,38541.20,38519.38,38533.44,0.026744


In [ ]:
df.isnull().sum()

open     0
high     0
low      0
close    0
vol      0
dtype: int64

### Feature add

In [ ]:
# Moving Average function
def moving_average(data, period):
    return data.rolling(window=period).mean()

In [ ]:
# Adding new columns for 10-day and 30-day moving averages
df['MA10'] = moving_average(df['close'], 10)
df['MA30'] = moving_average(df['close'], 30)

In [ ]:
df

,open,high,low,close,vol,MA10,MA30
date,,,,,,,
2021-08-01 00:01:00,41469.05,41469.05,41332.91,41348.38,3.513764,NaN,NaN
2021-08-01 00:02:00,41348.38,41348.38,41165.96,41196.18,8.040649,NaN,NaN
2021-08-01 00:03:00,41196.18,41339.38,41183.56,41327.59,3.081120,NaN,NaN
2021-08-01 00:04:00,41327.59,41387.71,41327.59,41379.84,0.044790,NaN,NaN
2021-08-01 00:05:00,41379.84,41550.50,41379.84,41550.50,0.196676,NaN,NaN
...,...,...,...,...,...,...,...
2022-01-31 23:55:00,38479.35,38510.17,38479.35,38510.17,0.008319,38442.754,38449.889333
2022-01-31 23:56:00,38510.17,38561.36,38506.33,38519.38,4.325445,38455.803,38452.091667
2022-01-31 23:57:00,38519.38,38541.20,38519.38,38533.44,0.026744,38470.501,38454.850000


In [ ]:
df = df[df.index > '2021-09-25']

In [ ]:
# Prepare the volume and price differences, normalize volume
BTC_vol = df["vol"].values
df_diff = df.diff().dropna()
df_diff["vol"] = np.log(1 + BTC_vol[:-1])

In [ ]:
df_aligned = df.loc[df_diff.index]

In [ ]:
# Train data
# Period : From start of 2018 to end of 2022
mask_train = (df_diff.index >= "2021-10-01") & (df_diff.index < "2021-12-31")
df_train = df_diff.loc[mask_train].copy()
train_close = df_aligned.loc[mask_train, "close"].values
df_train["Relative_Close"] = train_close / train_close[0]

In [ ]:
# Test data
# Period : Whole 2023
mask_test = (df_diff.index >= "2021-12-31") & (df_diff.index < "2022-01-01")  # December 2018 for testing
df_test = df_diff.loc[mask_test].copy()
test_close = df_aligned.loc[mask_test, "close"].values
df_test["Relative_Close"] = test_close / train_close[0]


In [ ]:
df_train.head()

,open,high,low,close,vol,MA10,MA30,Relative_Close
date,,,,,,,,
2021-10-01 00:00:00,49.39,0.00,35.28,0.00,0.172721,1.982,1.288000,1.000000
2021-10-01 00:01:00,0.00,43.23,14.11,41.79,0.138222,6.593,1.023000,1.000953
2021-10-01 00:02:00,41.79,85.12,-27.83,86.56,1.004677,11.712,3.451333,1.002928
2021-10-01 00:03:00,86.56,16.26,127.39,14.35,2.271787,18.638,2.855333,1.003256
2021-10-01 00:04:00,14.35,12.35,24.08,14.26,1.217967,18.893,4.195667,1.003581


In [ ]:
df_test.head()

,open,high,low,close,vol,MA10,MA30,Relative_Close
date,,,,,,,,
2021-12-31 00:00:00,-8.94,-23.58,-30.65,-30.65,0.108346,-1.422,-1.886000,1.074458
2021-12-31 00:01:00,-30.65,67.03,0.00,95.45,0.035732,5.754,0.532333,1.076636
2021-12-31 00:02:00,95.45,76.63,88.21,72.93,0.090812,11.848,5.238333,1.078300
2021-12-31 00:03:00,72.93,19.19,27.12,-53.05,0.887851,5.516,2.523000,1.077090
2021-12-31 00:04:00,-53.05,-58.71,-2.89,19.46,0.722145,5.572,3.719667,1.077534


In [ ]:
# Generate dataset function
def generate_dataset(df, seq_len):
    X_list, y_list = [], []
    for i in range(len(df) - seq_len):
        X_list.append(df.iloc[i:(i+seq_len), :].values)
        y_list.append(df["close"].iloc[i + seq_len])
    return np.array(X_list), np.array(y_list)

In [ ]:
LAG = 5

In [ ]:
validation_size = 0.2
n_validation = int(len(df_train) * validation_size)

df_val = df_train.iloc[-n_validation:]
df_train_reduced = df_train.iloc[:-n_validation]

X_train, y_train = generate_dataset(df_train_reduced, LAG)
X_val, y_val = generate_dataset(pd.concat((df_train_reduced.iloc[-LAG:], df_val)), LAG)
X_test, y_test = generate_dataset(pd.concat((df_train.iloc[-LAG:], df_test)), LAG)

In [ ]:
num_samples, num_timesteps, num_features = X_train.shape
X_train_reshaped = X_train.reshape(-1, num_features)
scaler = MinMaxScaler(feature_range=(0, 1))
X_train_scaled = scaler.fit_transform(X_train_reshaped)
X_train_scaled = X_train_scaled.reshape(num_samples, num_timesteps, num_features)

In [ ]:
num_samples_val, num_timesteps, num_features = X_val.shape
X_val_reshaped = X_val.reshape(-1, num_features)
X_val_scaled = scaler.transform(X_val_reshaped)
X_val_scaled = X_val_scaled.reshape(num_samples_val, num_timesteps, num_features)

In [ ]:
num_samples_test, num_timesteps, num_features = X_test.shape
X_test_reshaped = X_test.reshape(-1, num_features)
X_test_scaled = scaler.transform(X_test_reshaped)
X_test_scaled = X_test_scaled.reshape(num_samples_test, num_timesteps, num_features)

### Hyperparameter Tuning

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from kerastuner.tuners import RandomSearch

<ipython-input-29-bf4685511596>:5: DeprecationWarning: `import kerastuner` is deprecated, please use `import keras_tuner`.
  from kerastuner.tuners import RandomSearch


In [ ]:
def build_model(hp):
    model = Sequential([
        LSTM(
            units=hp.Int('units1', min_value=32, max_value=256, step=32),
            return_sequences=True,
            input_shape=(X_train.shape[1], X_train.shape[2]),
            recurrent_dropout=hp.Float('recurrent_dropout1', min_value=0.0, max_value=0.5, step=0.1)
        ),
        Dropout(rate=hp.Float('dropout1', min_value=0.0, max_value=0.5, step=0.1)),
        LSTM(
            units=hp.Int('units2', min_value=32, max_value=256, step=32),
            recurrent_dropout=hp.Float('recurrent_dropout2', min_value=0.0, max_value=0.5, step=0.1)
        ),
        Dropout(rate=hp.Float('dropout2', min_value=0.0, max_value=0.5, step=0.1)),
        Dense(
            units=hp.Int('dense_units', min_value=16, max_value=128, step=16),
            activation='relu'
        ),
        Dense(1)
    ])

    model.compile(
        optimizer=Adam(
            learning_rate=hp.Float('learning_rate', min_value=1e-4, max_value=1e-2, sampling='LOG')
        ),
        loss='mse'
    )

    return model

In [ ]:
tuner = RandomSearch(
    build_model,
    objective='val_loss',
    max_trials=10,  # Number of different configurations to try
    executions_per_trial=1,  # Number of models to train for each trial
    directory='my_dir',  # Directory to save logs and models
    project_name='lstm_tuning'
)

In [ ]:
# Display search space summary
tuner.search_space_summary()

Search space summary
Default search space size: 8
units1 (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 256, 'step': 32, 'sampling': 'linear'}
recurrent_dropout1 (Float)
{'default': 0.0, 'conditions': [], 'min_value': 0.0, 'max_value': 0.5, 'step': 0.1, 'sampling': 'linear'}
dropout1 (Float)
{'default': 0.0, 'conditions': [], 'min_value': 0.0, 'max_value': 0.5, 'step': 0.1, 'sampling': 'linear'}
units2 (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 256, 'step': 32, 'sampling': 'linear'}
recurrent_dropout2 (Float)
{'default': 0.0, 'conditions': [], 'min_value': 0.0, 'max_value': 0.5, 'step': 0.1, 'sampling': 'linear'}
dropout2 (Float)
{'default': 0.0, 'conditions': [], 'min_value': 0.0, 'max_value': 0.5, 'step': 0.1, 'sampling': 'linear'}
dense_units (Int)
{'default': None, 'conditions': [], 'min_value': 16, 'max_value': 128, 'step': 16, 'sampling': 'linear'}
learning_rate (Float)
{'default': 0.0001, 'conditions': [], 'min_value': 0.0001

In [ ]:
# Perform the hyperparameter search
tuner.search(
    X_train_scaled, y_train,
    epochs=5,
    validation_data=(X_val_scaled, y_val),
    callbacks=[tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3)]
)

Trial 10 Complete [00h 06m 06s]
val_loss: 1554.7833251953125

Best val_loss So Far: 1554.7626953125
Total elapsed time: 01h 40m 35s


In [ ]:
# Initialize an empty list to hold each trial's data
trial_data = []

# Iterate through each trial and collect the data
for trial_id, trial in tuner.oracle.trials.items():
    if trial.status == "COMPLETED":
        # Extract the final validation loss for the trial
        val_loss = trial.metrics.get_best_value('val_loss')
        # Prepare a dictionary for the trial
        trial_info = {
            'Trial ID': trial_id,
            'MSE': val_loss
        }
        # Update the dictionary with the hyperparameters
        trial_info.update(trial.hyperparameters.values)
        # Append the dictionary to the list
        trial_data.append(trial_info)

# Convert the list of dictionaries to a DataFrame
df_trials = pd.DataFrame(trial_data)

# Display the DataFrame
df_trials

,Trial ID,MSE,units1,recurrent_dropout1,dropout1,units2,recurrent_dropout2,dropout2,dense_units,learning_rate
0,00,1554.773315,128,0.1,0.3,192,0.0,0.2,112,0.001735
1,01,1554.785767,224,0.2,0.1,128,0.2,0.0,16,0.000513
2,02,1554.772095,192,0.1,0.3,224,0.2,0.0,112,0.005689
3,03,1554.773071,96,0.0,0.4,128,0.0,0.3,64,0.007101
4,04,1554.778564,256,0.2,0.3,96,0.2,0.2,32,0.007836
5,05,1554.791992,96,0.1,0.0,256,0.4,0.0,64,0.001413
6,06,1554.806519,96,0.4,0.4,192,0.3,0.3,16,0.003580
7,07,1554.762695,96,0.1,0.0,96,0.4,0.3,128,0.000246
8,08,1554.769043,96,0.4,0.2,32,0.2,0.3,128,0.000739
9,09,1554.783325,32,0.3,0.0,160,0.4,0.1,48,0.003748


In [ ]:
# Get the optimal hyperparameters
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]

print(f"""
The hyperparameter search is complete. The optimal number of units in the first LSTM layer is {best_hps.get('units1')},
the optimal dropout rates are {best_hps.get('dropout1')} for the first dropout layer and {best_hps.get('dropout2')} for the second dropout layer,
the optimal number of units in the second LSTM layer is {best_hps.get('units2')}, and the optimal learning rate for the optimizer
is {best_hps.get('learning_rate')}.
""")


The hyperparameter search is complete. The optimal number of units in the first LSTM layer is 96,
the optimal dropout rates are 0.0 for the first dropout layer and 0.30000000000000004 for the second dropout layer,
the optimal number of units in the second LSTM layer is 96, and the optimal learning rate for the optimizer
is 0.0002461390263263757.



In [ ]:
model = tuner.hypermodel.build(best_hps)
history = model.fit(X_train_scaled, y_train, epochs=50, validation_data=(X_val_scaled, y_val))

Epoch 1/50
3276/3276 [==============================] - 79s 23ms/step - loss: 3031.9121 - val_loss: 1554.8306
Epoch 2/50
3276/3276 [==============================] - 73s 22ms/step - loss: 3031.8748 - val_loss: 1554.8961
Epoch 3/50
3276/3276 [==============================] - 72s 22ms/step - loss: 3031.8772 - val_loss: 1554.7933
Epoch 4/50
3276/3276 [==============================] - 71s 22ms/step - loss: 3031.8523 - val_loss: 1554.8502
Epoch 5/50
3276/3276 [==============================] - 72s 22ms/step - loss: 3031.8613 - val_loss: 1554.8148
Epoch 6/50
3276/3276 [==============================] - 72s 22ms/step - loss: 3031.8142 - val_loss: 1554.9871
Epoch 7/50
3276/3276 [==============================] - 73s 22ms/step - loss: 3031.8762 - val_loss: 1554.8915
Epoch 8/50
3276/3276 [==============================] - 75s 23ms/step - loss: 3031.8506 - val_loss: 1554.8489
Epoch 9/50
3276/3276 [==============================] - 72s 22ms/step - loss: 3031.6602 - val_loss: 1554.7008
Epoch 10/5

In [ ]:
predicted_prices = model.predict(X_test_scaled)

45/45 [==============================] - 1s 5ms/step


In [ ]:
print("Test MSE:", np.mean((predicted_prices - y_test)**2))